## Example to load SWOT dataset (netcdf)

In [3]:
import s3fs
import sys
from itertools import product
import xarray as xr
import intake
import yaml
import netCDF4 

    
cycle_number=510
base_s3_folder= "s3://project-meom-ige/SWOT_L3_V1_0"
s3_folder = f"{base_s3_folder}/cycle_{cycle_number}/"
endpoint_url= 'https://minio.lab.dive.edito.eu/'
s3 = s3fs.S3FileSystem(anon=True,client_kwargs={"endpoint_url": endpoint_url})

# List NetCDF files in the folder
netcdf_files = [(endpoint_url+file) for file in s3.glob(f"{base_s3_folder}/cycle_{cycle_number}/*.nc")]

if not netcdf_files:
    raise ValueError("No NetCDF files found in the specified S3 location.")


# 🔹 Generate Intake catalog YAML with correct `storage_options`
#intake_xarray.NetCDFSource
catalog_dict = {
    "sources": {
        "SWOT": {
            "description": "SWOT dataset load ",
            "driver": "netcdf",
            "args": {
                "urlpath": netcdf_files,  # Explicit list of S3 URLs
                "engine": "h5netcdf",
                "chunks": {}
            }
        }
    }
}



# Save YAML catalog
with open("testcatalog.yaml", "w") as yaml_file:
    yaml.dump(catalog_dict, yaml_file, default_flow_style=False)


In [5]:
cat = intake.open_catalog("testcatalog.yaml")
raw_entry = cat._entries['SWOT'].describe()
url_list = raw_entry["args"]["urlpath"]
single_url = url_list[0]
new_source = intake.open_netcdf(urlpath=single_url,engine="h5netcdf")
ds = new_source.read()  # or .to_dask() if it's large

In [6]:
ds

<xarray.Dataset> Size: 27MB
Dimensions:         (num_lines: 9860, num_pixels: 69, num_nadir: 0)
Coordinates:
    latitude        (num_lines, num_pixels) float64 5MB ...
    longitude       (num_lines, num_pixels) float64 5MB ...
Dimensions without coordinates: num_lines, num_pixels, num_nadir
Data variables:
    time            (num_lines) datetime64[ns] 79kB ...
    mdt             (num_lines, num_pixels) float64 5MB ...
    ssha            (num_lines, num_pixels) float64 5MB ...
    ssha_noiseless  (num_lines, num_pixels) float64 5MB ...
    i_num_line      (num_nadir) int16 0B ...
    i_num_pixel     (num_nadir) int8 0B ...
Attributes: (12/42)
    Conventions:                     CF-1.7
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea Surface Height measured by Altimetry
    geospatial_lat_units:            degrees_north
    geospatial_lon_units:            degrees_east
    ...                              ...
    geospatial_lat_min:              -78.272196
    geospatial_lat_max:              78.272247
    geospatial_lon_min:              8.4e-05
    geospatial_lon_max:              359.999504
    data_used:                       SWOT KaRIn L2_LR_SSH PGC0 (NASA/CNES). D...
    doi:                             https://doi.org/10.24400/527896/A01-2023...